In [1]:
# NOTE: If you are running this notebook on Google Colab,
#       then uncomment the two lines below and then run this cell!

!pip install datasets evaluate --upgrade
!python -m spacy download de_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 14.5 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 34.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the pac

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
import spacy
import datasets
import torchtext
import tqdm
import evaluate
import pandas as pd

# from datasets import Dataset, DatasetDict
from datasets import load_dataset, Dataset, DatasetDict



We'll set all possible random seeds for deterministic results.


In [3]:
seed = 1234

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [3]:
import numpy as np
np.exp(5.6)

270.42640742615254

In [25]:

def load_custom_dataset(path):
    df = pd.read_csv(path)
    english_sentences = df['en']


    zulu_sentences = df['zu']

    # Ensure both files have the same number of lines
    assert len(english_sentences) == len(zulu_sentences), "Mismatch in number of lines between the files."

    # Create a dictionary with the data
    data_dict = {'en': english_sentences, 'de': zulu_sentences}

    # Convert dictionary into a Hugging Face dataset
    dataset = Dataset.from_dict(data_dict)

    return dataset

# Example paths
path = '/content/drive/MyDrive/en-zu.training.csv'
path_val='/content/drive/MyDrive/en-zu.eval.csv'
# df = pd.read_csv(path)
# df
# Load dataset
train_dataset = load_custom_dataset(path)
valid_dataset = load_custom_dataset(path_val)
test_dataset = load_custom_dataset(path_val)

dataset = DatasetDict({
    "train": train_dataset,
    "validation": valid_dataset,
    "test": test_dataset
})

# Print some examples
print("Example from the dataset:")
print(train_dataset[0])


Example from the dataset:
{'en': "His solo albums in the 60s were some of the most brilliant and breathtaking beautiful sounds I've ever heard.", 'de': 'Ama-albhamu akhe e-solo eminyaka yawo-60 angamanye anezingoma ezimnandi kakhulu kwengake ngazizwa.'}


In [26]:
print(dataset)
# Print example from the training set
print("Example from the training set:")
print(dataset["train"][0])

# Print example from the validation set
print("\nExample from the validation set:")
print(dataset["validation"][0])

# Print example from the test set
print("\nExample from the test set:")
print(dataset["test"][0])


DatasetDict({
    train: Dataset({
        features: ['en', 'de'],
        num_rows: 4739
    })
    validation: Dataset({
        features: ['en', 'de'],
        num_rows: 998
    })
    test: Dataset({
        features: ['en', 'de'],
        num_rows: 998
    })
})
Example from the training set:
{'en': "His solo albums in the 60s were some of the most brilliant and breathtaking beautiful sounds I've ever heard.", 'de': 'Ama-albhamu akhe e-solo eminyaka yawo-60 angamanye anezingoma ezimnandi kakhulu kwengake ngazizwa.'}

Example from the validation set:
{'en': 'Peter Van Sant: And it means what?', 'de': 'Peter Van Sant: Bese kusho ukuthini?'}

Example from the test set:
{'en': 'Peter Van Sant: And it means what?', 'de': 'Peter Van Sant: Bese kusho ukuthini?'}


In [27]:
dataset

DatasetDict({
    train: Dataset({
        features: ['en', 'de'],
        num_rows: 4739
    })
    validation: Dataset({
        features: ['en', 'de'],
        num_rows: 998
    })
    test: Dataset({
        features: ['en', 'de'],
        num_rows: 998
    })
})

For convenience, we create a variable for each split. Each being a `Dataset` object.


In [28]:
train_data, valid_data, test_data = (
    dataset["train"],
    dataset["validation"],
    dataset["test"],
)

We can index into each `Dataset` to view an individual example. Each example has two features: "en" and "de", which are the parallel English and German sentences.


In [29]:
train_data[0]

{'en': "His solo albums in the 60s were some of the most brilliant and breathtaking beautiful sounds I've ever heard.",
 'de': 'Ama-albhamu akhe e-solo eminyaka yawo-60 angamanye anezingoma ezimnandi kakhulu kwengake ngazizwa.'}

In [30]:
en_nlp = spacy.load("en_core_web_sm")
de_nlp = spacy.load("de_core_news_sm")

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [31]:
def tokenize_example(example, en_nlp, de_nlp, max_length, lower, sos_token, eos_token):
    en_tokens = [token.text for token in en_nlp.tokenizer(example["en"])][:max_length]
    de_tokens = [token.text for token in de_nlp.tokenizer(example["de"])][:max_length]
    if lower:
        en_tokens = [token.lower() for token in en_tokens]
        de_tokens = [token.lower() for token in de_tokens]
    en_tokens = [sos_token] + en_tokens + [eos_token]
    de_tokens = [sos_token] + de_tokens + [eos_token]
    return {"en_tokens": en_tokens, "de_tokens": de_tokens}

In [32]:
max_length = 1_000
lower = True
sos_token = "<sos>"
eos_token = "<eos>"

fn_kwargs = {
    "en_nlp": en_nlp,
    "de_nlp": de_nlp,
    "max_length": max_length,
    "lower": lower,
    "sos_token": sos_token,
    "eos_token": eos_token,
}

train_data = train_data.map(tokenize_example, fn_kwargs=fn_kwargs)
valid_data = valid_data.map(tokenize_example, fn_kwargs=fn_kwargs)
test_data = test_data.map(tokenize_example, fn_kwargs=fn_kwargs)

Map:   0%|          | 0/4739 [00:00<?, ? examples/s]

Map:   0%|          | 0/998 [00:00<?, ? examples/s]

Map:   0%|          | 0/998 [00:00<?, ? examples/s]

In [33]:
min_freq = 2
unk_token = "<unk>"
pad_token = "<pad>"

special_tokens = [
    unk_token,
    pad_token,
    sos_token,
    eos_token,
]

en_vocab = torchtext.vocab.build_vocab_from_iterator(
    train_data["en_tokens"],
    min_freq=min_freq,
    specials=special_tokens,
)

de_vocab = torchtext.vocab.build_vocab_from_iterator(
    train_data["de_tokens"],
    min_freq=min_freq,
    specials=special_tokens,
)

In [36]:
de_vocab.get_itos()[:10]

['<unk>',
 '<pad>',
 '<sos>',
 '<eos>',
 ',',
 '.',
 '"',
 'ukuthi',
 'futhi',
 'kakhulu']

In [42]:
assert en_vocab[unk_token] == de_vocab[unk_token]
assert en_vocab[pad_token] == de_vocab[pad_token]

unk_index = en_vocab[unk_token]
pad_index = en_vocab[pad_token]

In [43]:
en_vocab.set_default_index(unk_index)
de_vocab.set_default_index(unk_index)

In [45]:
en_vocab.get_itos()[0]

'<unk>'

In [49]:
def numericalize_example(example, en_vocab, de_vocab):
    en_ids = en_vocab.lookup_indices(example["en_tokens"])
    de_ids = de_vocab.lookup_indices(example["de_tokens"])
    return {"en_ids": en_ids, "de_ids": de_ids}

In [50]:
fn_kwargs = {"en_vocab": en_vocab, "de_vocab": de_vocab}

train_data = train_data.map(numericalize_example, fn_kwargs=fn_kwargs)
valid_data = valid_data.map(numericalize_example, fn_kwargs=fn_kwargs)
test_data = test_data.map(numericalize_example, fn_kwargs=fn_kwargs)

Map:   0%|          | 0/4739 [00:00<?, ? examples/s]

Map:   0%|          | 0/998 [00:00<?, ? examples/s]

Map:   0%|          | 0/998 [00:00<?, ? examples/s]

In [ ]:
import sentencepiece as spm
spm.SentencePieceTrainer.train(input='/content/drive/MyDrive/dataset-en.txt', model_prefix='/content/drive/MyDrive/smp_en', vocab_size=10000)
spm.SentencePieceTrainer.train(input='/content/drive/MyDrive/dataset-zu.txt', model_prefix='/content/drive/MyDrive/smp_zu', vocab_size=20000)
import sentencepiece as spm

# Load SentencePiece model
sp = spm.SentencePieceProcessor()
spz = spm.SentencePieceProcessor()
sp.load('/content/drive/MyDrive/smp_en.model')
spz.load('/content/drive/MyDrive/smp_zu.model')


In [ ]:
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

#feature engineering
loaded_model = load_model("model_srl_BiLstm.h5")

for ele in train_data:
    # Tokenize the example sentence
    token = sp.texts_to_sequences([train_data['en']])

    # Pad the sequence to ensure it has the same length as the sequences used during training
    example_sequence_padded = pad_sequences(token, maxlen=max_length, padding='post')

    predicted_labels = np.argmax(loaded_model.predict(example_sequence_padded),axis=-1)

    train_data['en_ids'] = [[predicted_labels[i], train_data['en_ids'][i]] for i in range(train_data['en_ids'])]


In [51]:
train_data[0]

{'en': "His solo albums in the 60s were some of the most brilliant and breathtaking beautiful sounds I've ever heard.",
 'de': 'Ama-albhamu akhe e-solo eminyaka yawo-60 angamanye anezingoma ezimnandi kakhulu kwengake ngazizwa.',
 'en_tokens': ['<sos>',
  'his',
  'solo',
  'albums',
  'in',
  'the',
  '60s',
  'were',
  'some',
  'of',
  'the',
  'most',
  'brilliant',
  'and',
  'breathtaking',
  'beautiful',
  'sounds',
  'i',
  "'ve",
  'ever',
  'heard',
  '.',
  '<eos>'],
 'de_tokens': ['<sos>',
  'ama-albhamu',
  'akhe',
  'e-solo',
  'eminyaka',
  'yawo-60',
  'angamanye',
  'anezingoma',
  'ezimnandi',
  'kakhulu',
  'kwengake',
  'ngazizwa',
  '.',
  '<eos>'],
 'en_ids': [2,
  29,
  6902,
  3781,
  11,
  4,
  3742,
  54,
  86,
  10,
  4,
  108,
  2181,
  8,
  0,
  1663,
  3605,
  23,
  186,
  422,
  1094,
  6,
  3],
 'de_ids': [2, 5019, 278, 0, 989, 0, 0, 0, 1962, 9, 0, 7202, 5, 3]}

In [52]:
en_vocab.lookup_tokens(train_data[0]["en_ids"])

['<sos>',
 'his',
 'solo',
 'albums',
 'in',
 'the',
 '60s',
 'were',
 'some',
 'of',
 'the',
 'most',
 'brilliant',
 'and',
 '<unk>',
 'beautiful',
 'sounds',
 'i',
 "'ve",
 'ever',
 'heard',
 '.',
 '<eos>']

In [53]:
data_type = "torch"
format_columns = ["en_ids", "de_ids"]

train_data = train_data.with_format(
    type=data_type, columns=format_columns, output_all_columns=True
)

valid_data = valid_data.with_format(
    type=data_type,
    columns=format_columns,
    output_all_columns=True,
)

test_data = test_data.with_format(
    type=data_type,
    columns=format_columns,
    output_all_columns=True,
)

In [54]:
train_data[0]

{'en_ids': tensor([   2,   29, 6902, 3781,   11,    4, 3742,   54,   86,   10,    4,  108,
         2181,    8,    0, 1663, 3605,   23,  186,  422, 1094,    6,    3]),
 'de_ids': tensor([   2, 5019,  278,    0,  989,    0,    0,    0, 1962,    9,    0, 7202,
            5,    3]),
 'en': "His solo albums in the 60s were some of the most brilliant and breathtaking beautiful sounds I've ever heard.",
 'de': 'Ama-albhamu akhe e-solo eminyaka yawo-60 angamanye anezingoma ezimnandi kakhulu kwengake ngazizwa.',
 'en_tokens': ['<sos>',
  'his',
  'solo',
  'albums',
  'in',
  'the',
  '60s',
  'were',
  'some',
  'of',
  'the',
  'most',
  'brilliant',
  'and',
  'breathtaking',
  'beautiful',
  'sounds',
  'i',
  "'ve",
  'ever',
  'heard',
  '.',
  '<eos>'],
 'de_tokens': ['<sos>',
  'ama-albhamu',
  'akhe',
  'e-solo',
  'eminyaka',
  'yawo-60',
  'angamanye',
  'anezingoma',
  'ezimnandi',
  'kakhulu',
  'kwengake',
  'ngazizwa',
  '.',
  '<eos>']}

In [55]:
type(train_data[0]["en_ids"])

torch.Tensor

In [56]:
def get_collate_fn(pad_index):
    def collate_fn(batch):
        batch_en_ids = [example["en_ids"] for example in batch]
        batch_de_ids = [example["de_ids"] for example in batch]
        batch_en_ids = nn.utils.rnn.pad_sequence(batch_en_ids, padding_value=pad_index)
        batch_de_ids = nn.utils.rnn.pad_sequence(batch_de_ids, padding_value=pad_index)
        batch = {
            "en_ids": batch_en_ids,
            "de_ids": batch_de_ids,
        }
        return batch

    return collate_fn

In [57]:
def get_data_loader(dataset, batch_size, pad_index, shuffle=False):
    collate_fn = get_collate_fn(pad_index)
    data_loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        collate_fn=collate_fn,
        shuffle=shuffle,
    )
    return data_loader

In [58]:
batch_size = 128

train_data_loader = get_data_loader(train_data, batch_size, pad_index, shuffle=True)
valid_data_loader = get_data_loader(valid_data, batch_size, pad_index)
test_data_loader = get_data_loader(test_data, batch_size, pad_index)

In [59]:
class Encoder(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        # src = [src length, batch size]
        embedded = self.dropout(self.embedding(src))
        # embedded = [src length, batch size, embedding dim]
        outputs, (hidden, cell) = self.rnn(embedded)
        # outputs = [src length, batch size, hidden dim * n directions]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # outputs are always from the top hidden layer
        return hidden, cell

In [60]:
class Decoder(nn.Module):
    def __init__(self, output_dim, embedding_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(output_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout)
        self.fc_out = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell):
        # input = [batch size]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # n directions in the decoder will both always be 1, therefore:
        # hidden = [n layers, batch size, hidden dim]
        # context = [n layers, batch size, hidden dim]
        input = input.unsqueeze(0)
        # input = [1, batch size]
        embedded = self.dropout(self.embedding(input))
        # embedded = [1, batch size, embedding dim]
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        # output = [seq length, batch size, hidden dim * n directions]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # seq length and n directions will always be 1 in this decoder, therefore:
        # output = [1, batch size, hidden dim]
        # hidden = [n layers, batch size, hidden dim]
        # cell = [n layers, batch size, hidden dim]
        prediction = self.fc_out(output.squeeze(0))
        # prediction = [batch size, output dim]
        return prediction, hidden, cell

In [61]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        assert (
            encoder.hidden_dim == decoder.hidden_dim
        ), "Hidden dimensions of encoder and decoder must be equal!"
        assert (
            encoder.n_layers == decoder.n_layers
        ), "Encoder and decoder must have equal number of layers!"

    def forward(self, src, trg, teacher_forcing_ratio):
        # src = [src length, batch size]
        # trg = [trg length, batch size]
        # teacher_forcing_ratio is probability to use teacher forcing
        # e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        batch_size = trg.shape[1]
        trg_length = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        # tensor to store decoder outputs
        outputs = torch.zeros(trg_length, batch_size, trg_vocab_size).to(self.device)
        # last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # first input to the decoder is the <sos> tokens
        input = trg[0, :]
        # input = [batch size]
        for t in range(1, trg_length):
            # insert input token embedding, previous hidden and previous cell states
            # receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            # output = [batch size, output dim]
            # hidden = [n layers, batch size, hidden dim]
            # cell = [n layers, batch size, hidden dim]
            # place predictions in a tensor holding predictions for each token
            outputs[t] = output
            # decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            # get the highest predicted token from our predictions
            top1 = output.argmax(1)
            # if teacher forcing, use actual next token as next input
            # if not, use predicted token
            input = trg[t] if teacher_force else top1
            # input = [batch size]
        return outputs

In [62]:
input_dim = len(de_vocab)
output_dim = len(en_vocab)
encoder_embedding_dim = 256
decoder_embedding_dim = 256
hidden_dim = 512
n_layers = 2
encoder_dropout = 0.2
decoder_dropout = 0.2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

encoder = Encoder(
    input_dim,
    encoder_embedding_dim,
    hidden_dim,
    n_layers,
    encoder_dropout,
)

decoder = Decoder(
    output_dim,
    decoder_embedding_dim,
    hidden_dim,
    n_layers,
    decoder_dropout,
)

model = Seq2Seq(encoder, decoder, device).to(device)

In [63]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)


model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(9046, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.2)
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(7323, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.2)
    (fc_out): Linear(in_features=512, out_features=7323, bias=True)
    (dropout): Dropout(p=0.2, inplace=False)
  )
)

We can also count the number of parameters in our model.


In [64]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f"The model has {count_parameters(model):,} trainable parameters")

The model has 15,303,579 trainable parameters


### Optimizer

We define our optimizer, which we use to update our parameters in the training loop. Check out [this](http://ruder.io/optimizing-gradient-descent/) post for information about different optimizers. Here, we'll use Adam.


In [65]:
optimizer = optim.Adam(model.parameters())

In [66]:
criterion = nn.CrossEntropyLoss(ignore_index=pad_index)

In [67]:
def train_fn(
    model, data_loader, optimizer, criterion, clip, teacher_forcing_ratio, device
):
    model.train()
    epoch_loss = 0
    for i, batch in enumerate(data_loader):
        src = batch["de_ids"].to(device)
        trg = batch["en_ids"].to(device)
        # src = [src length, batch size]
        # trg = [trg length, batch size]
        optimizer.zero_grad()
        output = model(src, trg, teacher_forcing_ratio)
        # output = [trg length, batch size, trg vocab size]
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        # output = [(trg length - 1) * batch size, trg vocab size]
        trg = trg[1:].view(-1)
        # trg = [(trg length - 1) * batch size]
        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

In [68]:
def evaluate_fn(model, data_loader, criterion, device):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(data_loader):
            src = batch["de_ids"].to(device)
            trg = batch["en_ids"].to(device)
            # src = [src length, batch size]
            # trg = [trg length, batch size]
            output = model(src, trg, 0)  # turn off teacher forcing
            # output = [trg length, batch size, trg vocab size]
            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            # output = [(trg length - 1) * batch size, trg vocab size]
            trg = trg[1:].view(-1)
            # trg = [(trg length - 1) * batch size]
            loss = criterion(output, trg)
            epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

In [71]:
n_epochs = 20
clip = 1.0
teacher_forcing_ratio = 0.5

best_valid_loss = float("inf")

for epoch in tqdm.tqdm(range(n_epochs)):
    train_loss = train_fn(
        model,
        train_data_loader,
        optimizer,
        criterion,
        clip,
        teacher_forcing_ratio,
        device,
    )
    valid_loss = evaluate_fn(
        model,
        valid_data_loader,
        criterion,
        device,
    )
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), "/content/drive/MyDrive/En-zu-withoutSRL-model.pt")
    print(f"\tTrain Loss: {train_loss:7.3f} | Train PPL: {np.exp(train_loss):7.3f}")
    print(f"\tValid Loss: {valid_loss:7.3f} | Valid PPL: {np.exp(valid_loss):7.3f}")

  5%|▌         | 1/20 [00:34<10:56, 34.56s/it]

	Train Loss:   6.222 | Train PPL: 503.560
	Valid Loss:   6.085 | Valid PPL: 439.255


 10%|█         | 2/20 [01:09<10:22, 34.59s/it]

	Train Loss:   6.175 | Train PPL: 480.756
	Valid Loss:   6.098 | Valid PPL: 444.996


 15%|█▌        | 3/20 [01:43<09:44, 34.41s/it]

...

	Train Loss:   5.582 | Train PPL: 265.730
	Valid Loss:   6.094 | Valid PPL: 443.153


 95%|█████████▌| 19/20 [10:48<00:34, 34.03s/it]

	Train Loss:   5.580 | Train PPL: 265.154
	Valid Loss:   6.105 | Valid PPL: 448.266


100%|██████████| 20/20 [11:22<00:00, 34.14s/it]

	Train Loss:   5.527 | Train PPL: 251.318
	Valid Loss:   6.108 | Valid PPL: 449.291


In [72]:
model.load_state_dict(torch.load("tut1-model.pt"))

test_loss = evaluate_fn(model, test_data_loader, criterion, device)

print(f"| Test Loss: {test_loss:.3f} | Test PPL: {np.exp(test_loss):7.3f} |")

| Test Loss: 6.095 | Test PPL: 443.695 |


In [73]:
def translate_sentence(
    sentence,
    model,
    en_nlp,
    de_nlp,
    en_vocab,
    de_vocab,
    lower,
    sos_token,
    eos_token,
    device,
    max_output_length=25,
):
    model.eval()
    with torch.no_grad():
        if isinstance(sentence, str):
            tokens = [token.text for token in de_nlp.tokenizer(sentence)]
        else:
            tokens = [token for token in sentence]
        if lower:
            tokens = [token.lower() for token in tokens]
        tokens = [sos_token] + tokens + [eos_token]
        ids = de_vocab.lookup_indices(tokens)
        tensor = torch.LongTensor(ids).unsqueeze(-1).to(device)
        hidden, cell = model.encoder(tensor)
        inputs = en_vocab.lookup_indices([sos_token])
        for _ in range(max_output_length):
            inputs_tensor = torch.LongTensor([inputs[-1]]).to(device)
            output, hidden, cell = model.decoder(inputs_tensor, hidden, cell)
            predicted_token = output.argmax(-1).item()
            inputs.append(predicted_token)
            if predicted_token == en_vocab[eos_token]:
                break
        tokens = en_vocab.lookup_tokens(inputs)
    return tokens

In [74]:
sentence = test_data[0]["de"]
expected_translation = test_data[0]["en"]

sentence, expected_translation

('Peter Van Sant: Bese kusho ukuthini?', 'Peter Van Sant: And it means what?')

In [75]:
translation = translate_sentence(
    sentence,
    model,
    en_nlp,
    de_nlp,
    en_vocab,
    de_vocab,
    lower,
    sos_token,
    eos_token,
    device,
)

In [76]:
translation

['<sos>',
 'the',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.']

In [77]:
sentence = "Ein Mann sitzt auf einer Bank."

In [78]:
translation = translate_sentence(
    sentence,
    model,
    en_nlp,
    de_nlp,
    en_vocab,
    de_vocab,
    lower,
    sos_token,
    eos_token,
    device,
)

And we receive our translation, which is reasonably close.


In [79]:
translation

['<sos>',
 'the',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.']

In [80]:
translations = [
    translate_sentence(
        example["de"],
        model,
        en_nlp,
        de_nlp,
        en_vocab,
        de_vocab,
        lower,
        sos_token,
        eos_token,
        device,
    )
    for example in tqdm.tqdm(test_data)
]

100%|██████████| 998/998 [00:18<00:00, 54.91it/s]


In [81]:
bleu = evaluate.load("bleu")

In [82]:
predictions = [" ".join(translation[1:-1]) for translation in translations]

references = [[example["en"]] for example in test_data]

In [83]:
predictions[0], references[0]

('the <unk> <unk> <unk> <unk> <unk> <unk> . . . . . . . . . . . . . . . . .',
 ['Peter Van Sant: And it means what?'])

In [84]:
def get_tokenizer_fn(nlp, lower):
    def tokenizer_fn(s):
        tokens = [token.text for token in nlp.tokenizer(s)]
        if lower:
            tokens = [token.lower() for token in tokens]
        return tokens

    return tokenizer_fn

In [85]:
tokenizer_fn = get_tokenizer_fn(en_nlp, lower)

In [86]:
tokenizer_fn(predictions[0]), tokenizer_fn(references[0][0])

(['the',
  '<',
  'unk',
  '>',
  '<',
  'unk',
  '>',
  '<',
  'unk',
  '>',
  '<',
  'unk',
  '>',
  '<',
  'unk',
  '>',
  '<',
  'unk',
  '>',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.'],
 ['peter', 'van', 'sant', ':', 'and', 'it', 'means', 'what', '?'])

In [87]:
results = bleu.compute(
    predictions=predictions, references=references, tokenizer=tokenizer_fn
)

In [ ]:
results